In [13]:
!pip install transformers torch ipywidgets langdetect textblob nltk
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, pipeline
import ipywidgets as widgets
from IPython.display import display
from langdetect import detect
from textblob import TextBlob
import nltk
nltk.download('punkt')
from sklearn.feature_extraction.text import TfidfVectorizer

bart_model_name = 'facebook/bart-large-cnn'
bart_tokenizer = BartTokenizer.from_pretrained(bart_model_name)
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_name)

text_generation_pipeline = pipeline("text-generation", model='gpt2')

sentiment_analysis_pipeline = pipeline("sentiment-analysis")


ner_pipeline = pipeline("ner")

def summarize(text, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True):
    preprocess_text = text.strip().replace("\n", " ")
    bart_input_text = "summarize: " + preprocess_text

    tokenized_text = bart_tokenizer.encode(bart_input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = bart_model.generate(tokenized_text, max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams, early_stopping=early_stopping)
    output = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

def rephrase(text, max_length=150, num_beams=4, early_stopping=True):
    preprocess_text = text.strip().replace("\n", " ")
    bart_input_text = "paraphrase: " + preprocess_text

    tokenized_text = bart_tokenizer.encode(bart_input_text, return_tensors="pt", max_length=512, truncation=True)
    rephrased_ids = bart_model.generate(tokenized_text, max_length=max_length, num_beams=num_beams, early_stopping=early_stopping)
    output = bart_tokenizer.decode(rephrased_ids[0], skip_special_tokens=True)
    return output


def detect_language(text):
    language = detect(text)
    return language


def complete_text(text, max_length=50):
    completion = text_generation_pipeline(text, max_length=max_length, num_return_sequences=1)
    return completion[0]['generated_text']

def analyze_sentiment(text):
    analysis = sentiment_analysis_pipeline(text)
    return analysis[0]

def recognize_entities(text):
    entities = ner_pipeline(text)
    return entities

def extract_keywords(text):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform([text])
    indices = X[0].nonzero()[1]
    keywords = [vectorizer.get_feature_names_out()[i] for i in indices]
    return keywords

def main():
    option = widgets.RadioButtons(
        options=['summarize', 'rephrase', 'detect language', 'complete text', 'analyze sentiment', 'recognize entities',  'extract keywords'],
        description='Choose an option:',
        disabled=False
    )

    text_area = widgets.Textarea(
        value='',
        placeholder='Type your paragraph here...',
        description='Paragraph:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='200px')
    )

    button = widgets.Button(
        description='Generate',
        disabled=False,
        button_style='',
        tooltip='Click to generate output',
        icon='check'
    )

    output_area = widgets.Output()

    def on_button_click(b):
        with output_area:
            output_area.clear_output()
            task = option.value
            paragraph = text_area.value
            if paragraph.strip():
                if task == 'summarize':
                    result = summarize(paragraph)
                elif task == 'rephrase':
                    result = rephrase(paragraph)

                elif task == 'detect language':
                    result = detect_language(paragraph)

                elif task == 'complete text':
                    result = complete_text(paragraph)
                elif task == 'analyze sentiment':
                    result = analyze_sentiment(paragraph)

                elif task == 'recognize entities':
                    result = recognize_entities(paragraph)
                elif task == 'extract keywords':
                    result = extract_keywords(paragraph)
                    print("Welcome to Abdulelah's project for Artificial Intelligence")
                    print(f"\nOriginal Paragraph:\n{paragraph}\n")
                    print("Extracted Keywords:")
                    for keyword in result:
                        print(f"- {keyword}")
                    return
                print(f"\nOriginal Paragraph:\n{paragraph}\n")
                print(f"{task.capitalize()}d Output:\n{result}")
            else:
                print("No paragraph provided.")

    button.on_click(on_button_click)

    display(option, text_area, button, output_area)

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on

RadioButtons(description='Choose an option:', options=('summarize', 'rephrase', 'detect language', 'complete t…

Textarea(value='', description='Paragraph:', layout=Layout(height='200px', width='100%'), placeholder='Type yo…

Button(description='Generate', icon='check', style=ButtonStyle(), tooltip='Click to generate output')

Output()